In [ ]:
# | default_exp utils/normalizations

# Imports

In [ ]:
# | export


import torch
from einops import rearrange
from torch import nn

# Normalizations

### Layernorms

In [ ]:
# | export


class LayerNorm2D(nn.LayerNorm):
    def forward(self, input: torch.Tensor) -> torch.Tensor:
        input = rearrange(input, "b c y x -> b y x c")
        input = super().forward(input)
        input = rearrange(input, "b y x c -> b c y x")
        return input

In [ ]:
sample_input = torch.randn(2, 30, 20, 10)
test = LayerNorm2D(30)
output = test(sample_input)
test, output.shape

(LayerNorm2D((30,), eps=1e-05, elementwise_affine=True), torch.Size([2, 30, 20, 10]))

In [ ]:
# | export


class LayerNorm3D(nn.LayerNorm):
    def forward(self, input: torch.Tensor) -> torch.Tensor:
        input = rearrange(input, "b c z y x -> b z y x c")
        input = super().forward(input)
        input = rearrange(input, "b z y x c -> b c z y x")
        return input

### Misc

In [ ]:
# | export


class DyT(nn.Module):
    # As proposed in Transformers without Normalization: https://arxiv.org/pdf/2503.10622

    def __init__(self, normalized_shape: int | list[int], alpha0: float = 0.5):
        super().__init__()

        if isinstance(normalized_shape, int):
            normalized_shape = (normalized_shape,)

        self.normalized_shape = normalized_shape
        self.alpha0 = alpha0

        self.alpha = nn.Parameter(torch.tensor([alpha0]))
        self.weight = nn.Parameter(torch.ones(normalized_shape, dtype=torch.float32))
        self.bias = nn.Parameter(torch.zeros(normalized_shape, dtype=torch.float32))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = torch.tanh(self.alpha * x)
        x = x * self.weight + self.bias
        return x

    def extra_repr(self) -> str:
        return f"normalized_shape={self.normalized_shape}, alpha0={self.alpha0}"

In [ ]:
sample_input = 5 * torch.randn(1, 30, 20, 10)
test = DyT((20, 10))
output = test(sample_input)
test, output.shape, output.min(), output.max()


(
    DyT(normalized_shape=(20, 10), alpha0=0.5),
    torch.Size([1, 30, 20, 10]),
    tensor(-1.0000, grad_fn=<MinBackward1>),
    tensor(1., grad_fn=<MaxBackward1>)
)

# Architecture

In [ ]:
# | export


def get_norm_layer(normalization_name: str, *args, **kwargs):
    if normalization_name == "layernorm":
        norm_layer = nn.LayerNorm(*args, **kwargs)
    elif normalization_name == "layernorm2d":
        norm_layer = LayerNorm2D(*args, **kwargs)
    elif normalization_name == "layernorm3d":
        norm_layer = LayerNorm3D(*args, **kwargs)
    elif normalization_name == "batchnorm" or normalization_name == "batchnorm1d":
        norm_layer = nn.BatchNorm1d(*args, **kwargs)
    elif normalization_name == "batchnorm2d":
        norm_layer = nn.BatchNorm2d(*args, **kwargs)
    elif normalization_name == "batchnorm3d":
        norm_layer = nn.BatchNorm3d(*args, **kwargs)
    elif normalization_name == "dyt":
        norm_layer = DyT(*args, **kwargs)
    else:
        raise ValueError(f"Normalization {normalization_name} not implemented")

    return norm_layer

In [ ]:
get_norm_layer("layernorm", 10)

LayerNorm((10,), eps=1e-05, elementwise_affine=True)

# nbdev_export

In [ ]:
!nbdev_export